# 라이브러리

In [218]:
options(warn =-1, repr.plot.width=15, repr.plot.height=9)
# ## 패키지 설치
library(dplyr)
library(stringr)
library(ggplot2)
library(gridExtra)
library(scales)
library(randomForest)
library(lubridate)
library(caret)
library(e1071)
library(nnet)
library(class)


# 전처리

In [219]:
### .csv 불러오기
data <- read.csv("./data/refined_data_filtered_major5_subset.csv")

In [220]:
data$근무지역분류 <- as.character(data$근무지역분류)
data$근무지역분류 <- factor(data$근무지역분류)
data$장애유형 <- as.character(data$장애유형)
data$장애유형 <- factor(data$장애유형)
data$중증여부 <- as.character(data$중증여부)
data$중증여부 <- factor(data$중증여부)
data$근무지역 <- as.character(data$근무지역)
data$근무지역 <- factor(data$근무지역)
data$취업직종대분류 <- as.character(data$취업직종대분류)
data$취업직종대분류 <- factor(data$취업직종대분류)
data$연령대 <- as.character(data$연령대)
data$연령대 <- factor(data$연령대)
data$장애유형_중증여부 <- as.character(data$장애유형_중증여부)
data$장애유형_중증여부 <- factor(data$장애유형_중증여부)
data$월 <- as.character(data$월)
data$월 <- factor(data$월)

In [221]:
data$target<-(ifelse(data$취업직종대분류=="경영·행정·사무직","경영·행정·사무직",data$취업직종대분류))

data$target <- ifelse(data$취업직종대분류 %in% c("청소 및 기타 개인서비스직", "음식 서비스직", "미용·예식 서비스직", "영업·판매직", "스포츠·레크리에이션직","경호·경비직"), "서비스직", data$target)

data$target <- ifelse(grepl("^\\d+$", data$target), "기타", data$target)

In [222]:
table(data$target)


경영·행정·사무직             기타         서비스직 
            8961             7474             8544 

In [223]:
data <- subset(data, data$target == "기타")
table(data$target)


기타 
7474 

In [224]:
data$target<- ifelse(data$취업직종대분류=="제조 단순직","제조 단순직",data$취업직종대분류)
data$target <- ifelse(data$취업직종대분류 %in% c("보건·의료직","돌봄 서비스직(간병·육아)","사회복지·종교직"), "간호·보건 및 돌봄 서비스 관련 직군", data$target)
data$target <- ifelse(data$target %in% c("간호·보건 및 돌봄 서비스 관련 직군","제조 단순직"),  data$target,"전문·생산 및 정비 관련 직군")
table(data$target)


간호·보건 및 돌봄 서비스 관련 직군        전문·생산 및 정비 관련 직군 
                              2291                               2709 
                       제조 단순직 
                              2474 

In [225]:
# 불필요한 컬럼 제거

data_filtered <- subset(data, select = -c(취업일자,연령,장애유형,중증여부,근무지역))
sum(is.na(data_filtered))

[1] 0

In [226]:
# 연령대
data_filtered$age_20 <- data_filtered$연령대 == 1
data_filtered$age_30 <- data_filtered$연령대 == 2
data_filtered$age_40 <- data_filtered$연령대 == 3
data_filtered$age_50 <- data_filtered$연령대 == 4
data_filtered$age_60 <- data_filtered$연령대 == 5
data_filtered$age_70 <- data_filtered$연령대 == 6

In [227]:
# 장애유형_중증여부 바꾸기
data_filtered$시각장애_중증 <- data_filtered$장애유형_중증여부 == "시각장애-중증"
data_filtered$시각장애_경증 <- data_filtered$장애유형_중증여부 == "시각장애-경증"

data_filtered$지체장애_중증 <- data_filtered$장애유형_중증여부 == "지체장애-중증"
data_filtered$지체장애_경증 <- data_filtered$장애유형_중증여부 == "지체장애-경증"

data_filtered$지적장애_경증 <- data_filtered$장애유형_중증여부 == "지적장애-경증"
data_filtered$지적장애_중증 <- data_filtered$장애유형_중증여부 == "지적장애-중증"

data_filtered$청각장애_경증 <- data_filtered$장애유형_중증여부 == "청각장애-경증"
data_filtered$청각장애_중증 <- data_filtered$장애유형_중증여부 == "청각장애-중증"

data_filtered$정신장애_경증 <- data_filtered$장애유형_중증여부 == "정신장애-경증"
data_filtered$정신장애_중증 <- data_filtered$장애유형_중증여부 == "정신장애-중증"

In [228]:
#   근무지역 분류 바꾸기
data_filtered$경북 <- data_filtered$근무지역분류 == "경북"

data_filtered$강원 <- data_filtered$근무지역분류 == "강원"

data_filtered$전남 <- data_filtered$근무지역분류 == "전남"

data_filtered$충북 <- data_filtered$근무지역분류 == "충북"

data_filtered$경기 <- data_filtered$근무지역분류 == "경기"

data_filtered$인천 <- data_filtered$근무지역분류 == "인천"

data_filtered$서울 <- data_filtered$근무지역분류 == "서울"

data_filtered$울산 <- data_filtered$근무지역분류 == "울산"

data_filtered$대전 <- data_filtered$근무지역분류 == "대전"

data_filtered$부산 <- data_filtered$근무지역분류 == "부산"

data_filtered$전북 <- data_filtered$근무지역분류 == "전북"

data_filtered$광주 <- data_filtered$근무지역분류 == "광주"

data_filtered$충남 <- data_filtered$근무지역분류 == "충남"

data_filtered$대구 <- data_filtered$근무지역분류 == "대구"

data_filtered$경남 <- data_filtered$근무지역분류 == "경남"

data_filtered$제주 <- data_filtered$근무지역분류 == "제주"

data_filtered$세종 <- data_filtered$근무지역분류 == "세종"

data_filtered$경북 <- data_filtered$근무지역분류 == "경북"

In [229]:
# 월
data_filtered$Jan <- data_filtered$월  == 01
data_filtered$Feb <- data_filtered$월  == 02
data_filtered$Mar <- data_filtered$월  == 03
data_filtered$Apr <- data_filtered$월  == 04
data_filtered$May <- data_filtered$월  == 05
data_filtered$Jun <- data_filtered$월  == 06
data_filtered$Jul <- data_filtered$월  == 07
data_filtered$Aug <- data_filtered$월  == 08
data_filtered$Sep <- data_filtered$월  == 09
data_filtered$Oct <- data_filtered$월  == 10
data_filtered$Nov <- data_filtered$월  == 11
data_filtered$Dec <- data_filtered$월  == 12

In [230]:
str(data_filtered)
sum(is.na(data_filtered))

'data.frame':	7474 obs. of  51 variables:
 $ 취업직종대분류   : Factor w/ 21 levels "건설·채굴직",..: 9 9 9 9 19 9 9 9 10 10 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 3 4 5 3 6 5 5 1 2 3 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 2 8 2 7 2 2 2 2 5 8 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 6 10 1 1 1 1 1 1 1 6 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 4 1 1 1 1 1 1 1 1 1 ...
 $ target           : chr  "간호·보건 및 돌봄 서비스 관련 직군" "간호·보건 및 돌봄 서비스 관련 직군" "간호·보건 및 돌봄 서비스 관련 직군" "간호·보건 및 돌봄 서비스 관련 직군" ...
 $ age_20           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_40           : logi  TRUE FALSE FALSE TRUE FALSE FALSE ...
 $ age_50           : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ age_60           : logi  FALSE FALSE TRUE FALSE FALSE TRUE ...
 $ age_70           : logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ 시각장애_중증    : logi  TRUE FALSE TRUE FALSE TR

[1] 0

In [231]:
str(data_filtered)
sum(is.na(data_filtered))

'data.frame':	7474 obs. of  51 variables:
 $ 취업직종대분류   : Factor w/ 21 levels "건설·채굴직",..: 9 9 9 9 19 9 9 9 10 10 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 3 4 5 3 6 5 5 1 2 3 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 2 8 2 7 2 2 2 2 5 8 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 6 10 1 1 1 1 1 1 1 6 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 4 1 1 1 1 1 1 1 1 1 ...
 $ target           : chr  "간호·보건 및 돌봄 서비스 관련 직군" "간호·보건 및 돌봄 서비스 관련 직군" "간호·보건 및 돌봄 서비스 관련 직군" "간호·보건 및 돌봄 서비스 관련 직군" ...
 $ age_20           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_40           : logi  TRUE FALSE FALSE TRUE FALSE FALSE ...
 $ age_50           : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ age_60           : logi  FALSE FALSE TRUE FALSE FALSE TRUE ...
 $ age_70           : logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ 시각장애_중증    : logi  TRUE FALSE TRUE FALSE TR

[1] 0

In [232]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

In [233]:
str(data_filtered_subset)
sum(is.na(data_filtered_subset))

'data.frame':	7474 obs. of  47 variables:
 $ 취업직종대분류: Factor w/ 21 levels "건설·채굴직",..: 9 9 9 9 19 9 9 9 10 10 ...
 $ target        : chr  "간호·보건 및 돌봄 서비스 관련 직군" "간호·보건 및 돌봄 서비스 관련 직군" "간호·보건 및 돌봄 서비스 관련 직군" "간호·보건 및 돌봄 서비스 관련 직군" ...
 $ age_20        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_40        : logi  TRUE FALSE FALSE TRUE FALSE FALSE ...
 $ age_50        : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ age_60        : logi  FALSE FALSE TRUE FALSE FALSE TRUE ...
 $ age_70        : logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ 시각장애_중증 : logi  TRUE FALSE TRUE FALSE TRUE TRUE ...
 $ 시각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증 : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ 지체장애_경증 : logi  FALSE FALSE FALSE TRUE FALSE FALSE ...
 $ 지적장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_경증 : logi  FALSE FALSE FAL

[1] 0

In [234]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

In [235]:
str(data_filtered_subset)

'data.frame':	7474 obs. of  47 variables:
 $ 취업직종대분류: Factor w/ 21 levels "건설·채굴직",..: 9 9 9 9 19 9 9 9 10 10 ...
 $ target        : chr  "간호·보건 및 돌봄 서비스 관련 직군" "간호·보건 및 돌봄 서비스 관련 직군" "간호·보건 및 돌봄 서비스 관련 직군" "간호·보건 및 돌봄 서비스 관련 직군" ...
 $ age_20        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_40        : logi  TRUE FALSE FALSE TRUE FALSE FALSE ...
 $ age_50        : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ age_60        : logi  FALSE FALSE TRUE FALSE FALSE TRUE ...
 $ age_70        : logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ 시각장애_중증 : logi  TRUE FALSE TRUE FALSE TRUE TRUE ...
 $ 시각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증 : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ 지체장애_경증 : logi  FALSE FALSE FALSE TRUE FALSE FALSE ...
 $ 지적장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_경증 : logi  FALSE FALSE FAL

In [236]:
summary(data_filtered_subset)

                                 취업직종대분류    target         
 제조 단순직                            :2474   Length:7474       
 보건·의료직                            :1609   Class :character  
 인쇄·목재·공예 및 기타 설치·정비·생산직: 680   Mode  :character  
 돌봄 서비스직(간병·육아)               : 452                     
 전기·전자 설치·정비·생산직             : 361                     
 운전·운송직                            : 349                     
 (Other)                                :1549                     
   age_20          age_30          age_40          age_50       
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:4941      FALSE:6177      FALSE:6229      FALSE:6228     
 TRUE :2533      TRUE :1297      TRUE :1245      TRUE :1246     
                                                                
                                                                
                                                                
                                                                
   

# 변동사항

In [237]:
data_filtered_subset$target <- factor(data_filtered_subset$target)

In [238]:
data_filtered_subset <- subset(data_filtered_subset, select = -c(취업직종대분류))

In [239]:
str(data_filtered_subset)

'data.frame':	7474 obs. of  46 variables:
 $ target       : Factor w/ 3 levels "간호·보건 및 돌봄 서비스 관련 직군",..: 1 1 1 1 2 1 1 1 1 1 ...
 $ age_20       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_40       : logi  TRUE FALSE FALSE TRUE FALSE FALSE ...
 $ age_50       : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ age_60       : logi  FALSE FALSE TRUE FALSE FALSE TRUE ...
 $ age_70       : logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ 시각장애_중증: logi  TRUE FALSE TRUE FALSE TRUE TRUE ...
 $ 시각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증: logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ 지체장애_경증: logi  FALSE FALSE FALSE TRUE FALSE FALSE ...
 $ 지적장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 정신장애_경증: logi  FALSE FALSE FALS

In [240]:
lm(target~.,data_filtered_subset)


Call:
lm(formula = target ~ ., data = data_filtered_subset)

Coefficients:
      (Intercept)         age_20TRUE         age_30TRUE         age_40TRUE  
         2.612536          -0.150303          -0.097915          -0.040744  
       age_50TRUE         age_60TRUE         age_70TRUE  시각장애_중증TRUE  
        -0.112507          -0.081705                 NA          -1.167487  
시각장애_경증TRUE  지체장애_중증TRUE  지체장애_경증TRUE  지적장애_경증TRUE  
        -0.570345          -0.213846          -0.389169          -0.018550  
지적장애_중증TRUE  청각장애_경증TRUE  청각장애_중증TRUE  정신장애_경증TRUE  
        -0.033960          -0.209517           0.026828          -0.099593  
정신장애_중증TRUE           경북TRUE           강원TRUE           전남TRUE  
               NA          -0.188169          -0.226608          -0.294898  
         충북TRUE           경기TRUE           인천TRUE           서울TRUE  
        -0.146043          -0.077953          -0.032709          -0.261258  
         울산TRUE           대전TRUE           부산TRUE           전북TRUE  
     

In [274]:
summary(data_filtered_subset)

                                target       age_20          age_30       
 간호·보건 및 돌봄 서비스 관련 직군:2291   Mode :logical   Mode :logical  
 전문·생산 및 정비 관련 직군       :2709   FALSE:4941      FALSE:6177     
 제조 단순직                       :2474   TRUE :2533      TRUE :1297     
   age_40          age_50          age_60          age_70       
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:6229      FALSE:6228      FALSE:6527      FALSE:7268     
 TRUE :1245      TRUE :1246      TRUE :947       TRUE :206      
 시각장애_중증   시각장애_경증   지체장애_중증   지체장애_경증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:6368      FALSE:7171      FALSE:7095      FALSE:6250     
 TRUE :1106      TRUE :303       TRUE :379       TRUE :1224     
 지적장애_경증   지적장애_중증   청각장애_경증   청각장애_중증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:7468      FALSE:4414      FALSE:7205      FALSE:6865     
 TRUE :6         TRUE :3060      TRUE :269       TRUE :609     

# 시험준비

In [241]:
samp <- createDataPartition(data_filtered_subset$target, p=0.7, list=F)

#   학습용
data.tr <- data_filtered_subset[samp,]

#   검증용
data.test <- data_filtered_subset[-samp,]

In [242]:
#   KNN을 위한 Training Data 준비
data.training <- subset(data.tr, select = -target)
data.training.label <- data.tr$target

In [243]:
#   문제 및 정답 만들기
x   <- subset(data.test, select = -target)
y   <- data.test$target


# 학습시키기

In [261]:
#  랜덤포레스트 학습 시키기
rf <- randomForest(target ~., data.tr, ntree=50)

In [245]:
# SVM 학습시키기
modelsvm <- svm(target~.,data.tr)

In [263]:
# KNN 학습시키기 3,5,7,
knnmodel <- knn(train=data.training, test=x,cl=data.training.label,k=3)

In [262]:
# nnet 학습시키기
modelnnet <- nnet(target~.,data.tr,size =10)

# weights:  493


initial  value 5802.757329 
iter  10 value 4289.963232
iter  20 value 4176.967063
iter  30 value 4058.581629
iter  40 value 3940.136399
iter  50 value 3823.232752
iter  60 value 3770.818408
iter  70 value 3725.885861
iter  80 value 3679.866709
iter  90 value 3625.254342
iter 100 value 3563.838790
final  value 3563.838790 
stopped after 100 iterations


# 검증하기

In [264]:
#  Random Forest 검증하기
predrf <- predict(rf, x)

In [265]:
#  SVM 검증하기
predmodelsvm <- predict(modelsvm, x)

In [266]:
# KNN 검증하기
# mean(knnmodel==y) 

In [267]:
# nnet 검증하기
prednnet = predict(modelnnet, x,type = "class")    # class : 객관식, raw : 주관식

# 예측력 확인

In [268]:
#  RF 예측력
mean(predrf == y)

[1] 0.6340919

In [269]:
#  SVM 예측력
mean(predmodelsvm == y)

[1] 0.5845605

In [271]:
#  KNN 예측력
mean(knnmodel == y)

[1] 0.6055332

In [272]:
#  nnet 예측력
mean(prednnet == y)

[1] 0.5908077

# 오분류표

In [256]:
#  Random Forest 오분류표
table(predrf,y)

                                    y
predrf                               간호·보건 및 돌봄 서비스 관련 직군
  간호·보건 및 돌봄 서비스 관련 직군                                417
  전문·생산 및 정비 관련 직군                                       160
  제조 단순직                                                       110
                                    y
predrf                               전문·생산 및 정비 관련 직군 제조 단순직
  간호·보건 및 돌봄 서비스 관련 직군                          30          24
  전문·생산 및 정비 관련 직군                                513         234
  제조 단순직                                                269         484

In [257]:
#  SVM 오분류표
table(predmodelsvm,y)

                                    y
predmodelsvm                         간호·보건 및 돌봄 서비스 관련 직군
  간호·보건 및 돌봄 서비스 관련 직군                                342
  전문·생산 및 정비 관련 직군                                       190
  제조 단순직                                                       155
                                    y
predmodelsvm                         전문·생산 및 정비 관련 직군 제조 단순직
  간호·보건 및 돌봄 서비스 관련 직군                          23          21
  전문·생산 및 정비 관련 직군                                515         268
  제조 단순직                                                274         453

In [258]:
#  KNN 오분류표
# table(y, knnmodel)

In [259]:
# nnet 오분류표
table(prednnet,y)

                                    y
prednnet                             간호·보건 및 돌봄 서비스 관련 직군
  간호·보건 및 돌봄 서비스 관련 직군                                446
  전문·생산 및 정비 관련 직군                                       135
  제조 단순직                                                       106
                                    y
prednnet                             전문·생산 및 정비 관련 직군 제조 단순직
  간호·보건 및 돌봄 서비스 관련 직군                          97          66
  전문·생산 및 정비 관련 직군                                464         226
  제조 단순직                                                251         450

In [260]:
# 새로운 데이터에 대한 예측
predict(rf, (list(age_20=FALSE,age_30=FALSE,age_40=FALSE,age_50=FALSE,age_60=TRUE,age_70=FALSE,
            시각장애_중증=TRUE,시각장애_경증=FALSE,지체장애_경증=FALSE,지체장애_중증=FALSE,지적장애_경증=FALSE,지적장애_중증=FALSE,
            청각장애_경증=FALSE,청각장애_중증=FALSE,정신장애_경증=FALSE,정신장애_중증=FALSE,
            경북=TRUE,강원=FALSE,전남=FALSE,충북=FALSE,경기=FALSE,인천=FALSE,서울=FALSE,울산=FALSE,대전=FALSE,부산=FALSE,
            전북=FALSE,광주=FALSE,충남=FALSE,대구=FALSE,경남=FALSE,제주=FALSE,세종=FALSE,경북=FALSE,
            Jan=TRUE,Feb=FALSE,Mar=FALSE,Apr=FALSE,May=FALSE,Jun=FALSE,Jul=FALSE,Aug=FALSE,Sep=FALSE,
            Oct=FALSE,Nov=FALSE,Dec=FALSE)), type = "prob")

# pred_prob <- predict(rf, new_data, type = "prob")
# pred_prob <- predict(rf, new_data, type = "prob")

,간호·보건 및 돌봄 서비스 관련 직군,전문·생산 및 정비 관련 직군,제조 단순직
1,0.996,0.004,0
